In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
data = pd.read_csv('./retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [11]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [12]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [13]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [14]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


In [15]:
item_features = pd.read_csv('./product.csv')
item_features.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [16]:
user_features = pd.read_csv('./hh_demographic.csv')
user_features.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


### Бейзлайны

In [17]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [18]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### 1.1 Random recommendation

In [19]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [20]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 6.21 s, sys: 0 ns, total: 6.21 s
Wall time: 6.23 s


### 1.2 Popularity-based recommendation

In [21]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [22]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 72 ms, sys: 0 ns, total: 72 ms
Wall time: 71.2 ms


### Задание 1. Weighted Random Recommendation
Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж

 - Можно сэмплировать товары случайно, но пропорционально какому-либо весу
 - Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [7]:
def calc_log_item_weights(data):
    item_weights = data.groupby(by='item_id')['sales_value'] \
       .sum() \
       .reset_index() \
       .rename(columns={'sales_value':'weight'})
    
    item_weights['weight'] += 1
    item_weights['weight'] = np.log(item_weights['weight'])
    item_weights['weight'] /= item_weights['weight'].sum()
    
    return item_weights

In [8]:
log_item_weights = calc_log_item_weights(data)

In [9]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

In [23]:
%%time

result['random_recommendation_log_weight'] = result['user_id'].apply(lambda x: weighted_random_recommendation(log_item_weights))

CPU times: user 1.84 s, sys: 0 ns, total: 1.84 s
Wall time: 1.84 s


In [24]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_log_weight
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[959420, 13008249, 10285118, 8159624, 949958]","[6534178, 6533889, 1029743, 6534166, 1082185]","[895936, 9527261, 891584, 9575434, 892117]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6398312, 10255525, 1122092, 908145, 10457561]","[6534178, 6533889, 1029743, 6534166, 1082185]","[15629795, 13417616, 9707454, 5569983, 1122961]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6039652, 5995368, 10456173, 1068148, 140226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[867847, 1071555, 1085977, 15717058, 9575559]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[996218, 8293195, 1061746, 9837690, 15717040]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1509456, 5563875, 12649586, 15452409, 855936]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[900185, 1072901, 8160436, 12262908, 12456647]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1018711, 7147661, 969771, 1080014, 5569432]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

### Item-Item Recommender / ItemKNN

In [25]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [26]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [28]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999
data_train.head()

/home/alex/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [29]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [30]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [31]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.56 s, sys: 16 ms, total: 1.58 s
Wall time: 1.11 s


In [32]:
%%time

result['itemitem'] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 84 ms, sys: 4 ms, total: 88 ms
Wall time: 85.2 ms


In [33]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_log_weight,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[959420, 13008249, 10285118, 8159624, 949958]","[6534178, 6533889, 1029743, 6534166, 1082185]","[895936, 9527261, 891584, 9575434, 892117]","[999999, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6398312, 10255525, 1122092, 908145, 10457561]","[6534178, 6533889, 1029743, 6534166, 1082185]","[15629795, 13417616, 9707454, 5569983, 1122961]","[999999, 1082185, 981760, 1098066, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6039652, 5995368, 10456173, 1068148, 140226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[867847, 1071555, 1085977, 15717058, 9575559]","[999999, 1082185, 981760, 1127831, 995242]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[996218, 8293195, 1061746, 9837690, 15717040]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1509456, 5563875, 12649586, 15452409, 855936]","[999999, 1082185, 981760, 1127831, 995242]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[900185, 1072901, 8160436, 12262908, 12456647]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1018711, 7147661, 969771, 1080014, 5569432]","[999999, 1082185, 981760, 1127831, 1098066]"


### Косинусное сходство и CosineRecommender

In [34]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.55 s, sys: 24 ms, total: 1.58 s
Wall time: 1.08 s


In [35]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 84 ms, sys: 12 ms, total: 96 ms
Wall time: 94.4 ms


In [36]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_log_weight,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[959420, 13008249, 10285118, 8159624, 949958]","[6534178, 6533889, 1029743, 6534166, 1082185]","[895936, 9527261, 891584, 9575434, 892117]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6398312, 10255525, 1122092, 908145, 10457561]","[6534178, 6533889, 1029743, 6534166, 1082185]","[15629795, 13417616, 9707454, 5569983, 1122961]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6039652, 5995368, 10456173, 1068148, 140226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[867847, 1071555, 1085977, 15717058, 9575559]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[996218, 8293195, 1061746, 9837690, 15717040]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1509456, 5563875, 12649586, 15452409, 855936]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[900185, 1072901, 8160436, 12262908, 12456647]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1018711, 7147661, 969771, 1080014, 5569432]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]"


In [37]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.58 s, sys: 20 ms, total: 1.6 s
Wall time: 1.11 s


In [38]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

CPU times: user 92 ms, sys: 4 ms, total: 96 ms
Wall time: 95.5 ms


In [39]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_log_weight,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[959420, 13008249, 10285118, 8159624, 949958]","[6534178, 6533889, 1029743, 6534166, 1082185]","[895936, 9527261, 891584, 9575434, 892117]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6398312, 10255525, 1122092, 908145, 10457561]","[6534178, 6533889, 1029743, 6534166, 1082185]","[15629795, 13417616, 9707454, 5569983, 1122961]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6039652, 5995368, 10456173, 1068148, 140226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[867847, 1071555, 1085977, 15717058, 9575559]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[996218, 8293195, 1061746, 9837690, 15717040]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1509456, 5563875, 12649586, 15452409, 855936]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[900185, 1072901, 8160436, 12262908, 12456647]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1018711, 7147661, 969771, 1080014, 5569432]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]"


### own purchases

In [40]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.58 s, sys: 32 ms, total: 1.61 s
Wall time: 1.13 s


In [41]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)])

CPU times: user 88 ms, sys: 0 ns, total: 88 ms
Wall time: 91 ms


In [42]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)

    return precision

In [44]:
model_names = [m for m in list(result)[2:] ]

In [45]:
model_names

['random_recommendation',
 'popular_recommendation',
 'random_recommendation_log_weight',
 'itemitem',
 'cosine',
 'tfidf',
 'own_purchases']

In [59]:
k = 5
pd.DataFrame(
    [
        (model_name, result.apply(lambda row: precision_at_k(recommended_list=row[model_name], 
                                                             bought_list=row['actual'], 
                                                             k=k), 
                                                             axis=1).mean())
        for model_name in model_names
    ], 
    columns=['model name', f'precision@{k}']
)

/home/alex/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


,model name,precision@5
0,random_recommendation,0.000392
1,popular_recommendation,0.155240
2,random_recommendation_log_weight,0.001273
3,itemitem,0.136925
4,cosine,0.132909
5,tfidf,0.138981
6,own_purchases,0.201917


# Задание 3*. Улучшение бейзлайнов и ItemItem
 - Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
 - Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей  𝐾 

In [62]:
# Получаем список топ 5000 продаваемых идентификаторов товаров
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)['item_id'].tolist()

# Выбираем в качестве data_train товары из топ 5000
data_train_top5000 = data_train.loc[data_train['item_id'].isin(top_5000), :]

In [64]:
%%time

items = data_train_top5000.item_id.unique()

result['random_recommendation_top5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 420 ms, sys: 4 ms, total: 424 ms
Wall time: 424 ms


In [65]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train_top5000, n=5)

result['popular_recommendation_top5000'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 22.1 ms


In [66]:
%%time

log_item_weights = calc_log_item_weights(data_train_top5000)

result['random_recommendation_log_weight_top5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(log_item_weights))

CPU times: user 388 ms, sys: 4 ms, total: 392 ms
Wall time: 390 ms


In [67]:
model_names = [m for m in list(result)[2:] ]
k = 5
pd.DataFrame(
    [
        (model_name, result.apply(lambda row: precision_at_k(recommended_list=row[model_name], 
                                                             bought_list=row['actual'], 
                                                             k=k), 
                                                             axis=1).mean())
        for model_name in model_names
    ], 
    columns=['model name', f'precision@{k}']
)

/home/alex/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


,model name,precision@5
0,random_recommendation,0.000392
1,popular_recommendation,0.155240
2,random_recommendation_log_weight,0.001273
3,itemitem,0.136925
4,cosine,0.132909
5,tfidf,0.138981
6,own_purchases,0.201917
7,random_recommendation_top5000,0.004897
8,popular_recommendation_top5000,0.155240
9,random_recommendation_log_weight_top5000,0.007052


##### Метрики бейзлайнов улучшились 

In [68]:
for k in range(1, 10):
    model = ItemItemRecommender(K=k, num_threads=4)                            # k - ближайшие соседи
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)      
    recs = model.recommend(userid=userid_to_id[2],                             
                            user_items=csr_matrix(user_item_matrix).tocsr(),  
                            N=5,                                               # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
    result[f'itemitem_K={k}'] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [73]:
model_names = [m for m in list(result)[2:] ]
k = 5
final_metrics = pd.DataFrame(
    [
        (model_name, result.apply(lambda row: precision_at_k(recommended_list=row[model_name], 
                                                             bought_list=row['actual'], 
                                                             k=k), 
                                                             axis=1).mean())
        for model_name in model_names
    ], 
    columns=['model name', f'precision@{k}']
)
final_metrics.sort_values(by=f'precision@{k}', ascending=False)


/home/alex/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


,model name,precision@5
6,own_purchases,0.201917
8,popular_recommendation_top5000,0.155240
1,popular_recommendation,0.155240
18,itemitem_K=9,0.141430
17,itemitem_K=8,0.141430
16,itemitem_K=7,0.141430
5,tfidf,0.138981
3,itemitem,0.136925
13,itemitem_K=4,0.134868
12,itemitem_K=3,0.133105
